In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tqdm openpyxl anuvaad 
!pip install -U deep-translator

In [ ]:
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def out(ver,data):
    data.to_csv('translate_awards_v'+ver+'_Kaggle_out.csv')
    data.to_pickle('translate_awards_v'+ver+'_Kaggle_out.pickle')
    data.to_excel('translate_awards_v'+ver+'_Kaggle_out.xlsx')
def out_csv(ver,data):
    data.to_csv('renders_awards_v'+ver+'_Kaggle_out.csv')

In [ ]:
import re
from deep_translator import GoogleTranslator
from time import sleep
from anuvaad import Anuvaad
import random
anu = Anuvaad("english-telugu")

In [ ]:
def google_onebyone(para):
    try:
        if pd.isna(para) or type(para) == np.int64:
            return np.nan
        else:
            if len(para) != 0:
                para = para.replace('\n','')
                para = para.replace('\t','')
                para = para.replace('\u200c','')
                lis_para = [i.strip() for i in para.strip().split('.') if i != '']
                gle_lis_para = []
                for lis_para_line in lis_para:
                    if lis_para_line:
                        gle_lis_para.append(GoogleTranslator(source='auto', target='te').translate(lis_para_line))
                if len(gle_lis_para) != 0: 
                    trans_para = '.'.join(gle_lis_para) + '.'
                trans_para = trans_para.replace('\u200c','')
                return trans_para
    except Exception:
        return -1
    return np.nan

def anu_vaad_onebyone(para):
    if len(para) != 0:
        lis_para = [i.strip() for i in para.strip().split('.') if i != '']
        anu_lis_para =  anu.anuvaad(lis_para)
        trans_para = '.'.join(anu_lis_para) + '.'
        return trans_para
    return np.nan

def clean_format(text):
    text  =text.replace('..','.')
    text = text.replace('మరియు',",")
    text = text.replace('యొక్క',",")
    text = text.replace('అతను',"ఇతను")
    text = text.replace('ఆమె',"ఈమె")
    text = text.replace(',',", ")
    text = text.replace('.',". ")
    text = text.replace('..',".")
    text = text.replace('..',".")
    text = text.replace(',.',".")
    text = text.replace('.,',".")
    text = text.replace('..','.')
    text = text.replace(',.',".")
    text = text.replace('.,',".")
    text = text.replace(' . ',". ")
    text = text.replace(' , ',", ")
    text = text.replace(' . ',". ")
    text = text.replace(' , ',", ")
    text = text.replace('  '," ")
    return text

def create_telugu_google_by_split(col,data,f,t):
    data[col+' Telugu'] = np.nan
    for i in tqdm(range(f,t),col):
        try:
#             sleep(random.randint(0,2))
            if not pd.isna(data[col][i]):
                wer = google_onebyone(data[col][i])
                if wer != -1:
                    data[col + ' Telugu'][i] = clean_format(wer)
                else:
                    data[col + ' Telugu'][i] = clean_format(anu_vaad_onebyone(data[col][i]))
        except Exception as e:
            print('!'+'-'*20+'ERR')
            print(e)
    return data

def trans_col_steps(col,s,awards):
    print('*'*30+'Translate Google '+col+'*'*30)
    step = s
    ci = 0 
    try:
        while ci + step < awards.shape[0] :
            awards = create_telugu_google_by_split(col,awards,ci,ci+step)
            ci += step
        awards = create_telugu_google_by_split(col,awards,ci,awards.shape[0])
    except Exception as e:
        print(e)
    d2 = {}
    for k,v in enumerate(awards.columns):
        d2[v] = k
    out('6_'+str(d2[col])+col,awards)
    print('$'*30+'FIN :: Translate Google '+ col +'$'*30)

In [ ]:
awards = pd.read_csv('/kaggle/input/padmaawardswiki/renders_awards_v4_Kaggle_out.csv',low_memory=False)
awards.head(10)

In [ ]:
trans_col_steps('Summary',50,awards)
# trans_col_steps('career',500,awards)
# trans_col_steps('biography',500,awards)
# trans_col_steps('early life',500,awards)
# trans_col_steps('awards',500,awards)
# trans_col_steps('personal life',500,awards)
# trans_col_steps('early life and education',500,awards)
# trans_col_steps('awards and honours',500,awards)
# trans_col_steps('death',500,awards)
# trans_col_steps('further reading',500,awards)
# trans_col_steps('bibliography',500,awards)
# trans_col_steps('legacy',500,awards)
# trans_col_steps('filmography',500,awards)
# trans_col_steps('awards and recognitions',500,awards)
# trans_col_steps('awards and recognition',500,awards)
# trans_col_steps('works',500,awards)
# trans_col_steps('publications',500,awards)
# trans_col_steps('education',500,awards)
# trans_col_steps('awards and honors',500,awards)
# trans_col_steps('notes',500,awards)
# trans_col_steps('books',500,awards)
# trans_col_steps('life',500,awards)
# trans_col_steps('early life and background',500,awards)
# trans_col_steps('recognition',500,awards)
# trans_col_steps('honours',500,awards)
# trans_col_steps('political career',500,awards)
# trans_col_steps('discography',500,awards)
# trans_col_steps('selected bibliography',500,awards)
# trans_col_steps('family',500,awards)
# trans_col_steps('positions',500,awards)
# trans_col_steps('in popular culture',500,awards)
# trans_col_steps('early life and career',500,awards)
# trans_col_steps('life and career',500,awards)
# trans_col_steps('honours and awards',500,awards)
# trans_col_steps('positions held',500,awards)
# trans_col_steps('achievements',500,awards)
# trans_col_steps('controversies',500,awards)
# trans_col_steps('literary works',500,awards)
# trans_col_steps('controversy',500,awards)
# trans_col_steps('writings',500,awards)
# trans_col_steps('death and legacy',500,awards)
# trans_col_steps('honors and awards',500,awards)
# trans_col_steps('work',500,awards)
# trans_col_steps('literary career',500,awards)
# trans_col_steps('later life',500,awards)
# trans_col_steps('film career',500,awards)
# trans_col_steps('early life and training',500,awards)
# trans_col_steps('selected publications',500,awards)
# trans_col_steps('philanthropy',500,awards)
# trans_col_steps('honors',500,awards)
# trans_col_steps('early years',500,awards)
# trans_col_steps('selected works',500,awards)
# trans_col_steps('education and career',500,awards)
# trans_col_steps('acting career',500,awards)
# trans_col_steps('early life and family',500,awards)
# trans_col_steps('academic career',500,awards)
# trans_col_steps('awards and nominations',500,awards)
# trans_col_steps('research',500,awards)
# trans_col_steps('early career',500,awards)
# trans_col_steps('politics',500,awards)
# trans_col_steps('illness and death',500,awards)
# trans_col_steps('accolades',500,awards)
# trans_col_steps('military career',500,awards)
# trans_col_steps('style',500,awards)
# trans_col_steps('early and personal life',500,awards)
# trans_col_steps('social work',500,awards)
# trans_col_steps('awards and accolades',500,awards)
# trans_col_steps('professional career',500,awards)
# trans_col_steps('musical career',500,awards)
# trans_col_steps('international career',500,awards)
# trans_col_steps('personal life and death',500,awards)
# trans_col_steps('exhibitions',500,awards)
# trans_col_steps('sources',500,awards)
# trans_col_steps('background',500,awards)
# trans_col_steps('activism',500,awards)
# trans_col_steps('later years',500,awards)
# trans_col_steps('contributions',500,awards)
# trans_col_steps('other work',500,awards)
# trans_col_steps('life sketch',500,awards)
# trans_col_steps('published works',500,awards)
# trans_col_steps('teaching',500,awards)
# trans_col_steps('professional life',500,awards)
# trans_col_steps('television',500,awards)
# trans_col_steps('notable works',500,awards)
# trans_col_steps('singing career',500,awards)
# trans_col_steps('award',500,awards)
# trans_col_steps('reception',500,awards)
# trans_col_steps('personal',500,awards)
# trans_col_steps('awards and achievements',500,awards)
# trans_col_steps('political life',500,awards)
# trans_col_steps('awards and decorations',500,awards)
# trans_col_steps('dates of rank',500,awards)
# trans_col_steps('major works',500,awards)
# trans_col_steps('playing style',500,awards)
# trans_col_steps('performing career',500,awards)
# trans_col_steps('disciples',500,awards)
# trans_col_steps('other activities',500,awards)
# trans_col_steps('quotes',500,awards)
# trans_col_steps('awards and distinctions',500,awards)
# trans_col_steps('awards and titles',500,awards)
# trans_col_steps('writing',500,awards)
# trans_col_steps('personal life and education',500,awards)
# trans_col_steps('music career',500,awards)
# trans_col_steps('students',500,awards)
# trans_col_steps('diplomatic career',500,awards)
# trans_col_steps('autobiography',500,awards)
# trans_col_steps('selected filmography',500,awards)
# trans_col_steps('literary contributions',500,awards)
# trans_col_steps('books authored',500,awards)
# trans_col_steps('biographies',500,awards)
# trans_col_steps('life and education',500,awards)
# trans_col_steps('in the media',500,awards)
# trans_col_steps('films',500,awards)
# trans_col_steps('poetry',500,awards)
# trans_col_steps('literary work',500,awards)
# trans_col_steps('coaching career',500,awards)
# trans_col_steps('domestic career',500,awards)
# trans_col_steps('training',500,awards)
# trans_col_steps('social service',500,awards)
# trans_col_steps('books and publications',500,awards)
# trans_col_steps('later career',500,awards)
# trans_col_steps('partial filmography',500,awards)
# trans_col_steps('other works',500,awards)
# trans_col_steps('family and personal life',500,awards)
# trans_col_steps('career highlights',500,awards)
# trans_col_steps('early days',500,awards)
# trans_col_steps('criticism',500,awards)
# trans_col_steps('business career',500,awards)
# trans_col_steps('memorials',500,awards)
# trans_col_steps('life and work',500,awards)
# trans_col_steps('contribution',500,awards)
# trans_col_steps('personal life and family',500,awards)
# trans_col_steps('political views',500,awards)
# trans_col_steps('records',500,awards)
# trans_col_steps('social activities',500,awards)
# trans_col_steps('plays',500,awards)
# trans_col_steps('selected articles',500,awards)
# trans_col_steps('playing career',500,awards)
# trans_col_steps('performances',500,awards)
# trans_col_steps('awards & honours',500,awards)
# trans_col_steps('cricket career',500,awards)
# trans_col_steps('memberships',500,awards)
# trans_col_steps('post retirement',500,awards)
# trans_col_steps('later life and death',500,awards)
# trans_col_steps('post-retirement',500,awards)
# trans_col_steps('list of indian writers',500,awards)
# trans_col_steps('accomplishments',500,awards)
# trans_col_steps('appearances in the following poetry anthologies',500,awards)
# trans_col_steps('footnotes',500,awards)
# trans_col_steps('freedom fighter',500,awards)
# trans_col_steps('television career',500,awards)
# trans_col_steps('philosophy',500,awards)
# trans_col_steps('posts held',500,awards)
# trans_col_steps('literature',500,awards)
# trans_col_steps('social activism',500,awards)
# trans_col_steps('collections',500,awards)
# trans_col_steps('later years and death',500,awards)
# trans_col_steps('recognitions',500,awards)
# trans_col_steps('popular culture',500,awards)
# trans_col_steps('international competitions',500,awards)
# trans_col_steps('affiliations',500,awards)
# trans_col_steps('book',500,awards)
# trans_col_steps('humanitarian work',500,awards)
# trans_col_steps('list of works',500,awards)
# trans_col_steps('birth',500,awards)
# trans_col_steps('captaincy',500,awards)
# trans_col_steps('media',500,awards)
# trans_col_steps('legacy and honours',500,awards)
# trans_col_steps('other achievements',500,awards)
# trans_col_steps('achievements and legacy',500,awards)
# trans_col_steps('honour',500,awards)
# trans_col_steps('filmography and accolades',500,awards)
# trans_col_steps('compositions',500,awards)
# trans_col_steps('commemoration',500,awards)
# trans_col_steps('concert performances',500,awards)
# trans_col_steps('performance timelines',500,awards)
# trans_col_steps('authorship',500,awards)
# trans_col_steps('childhood',500,awards)

In [ ]:
print('$'*50 , ' DONE ','$'*50)